
# Deploy FLAN-XXL Model with high performance on SageMaker using NVIDIA FasterTransformer

In this notebook, we explore how to host a FLAN-T5-XXL large language model with FP32 precision on SageMaker using the NVIDIA Faster Transformer. We use DJLServing as the model serving solution in this example. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent blog post (https://aws.amazon.com/blogs/machine-learning/deploy-bloom-176b-and-opt-30b-on-amazon-sagemaker-with-large-model-inference-deep-learning-containers-and-deepspeed/).


Model parallelism can help deploy large models that would normally be too large for a single GPU. With model parallelism, we partition and distribute a model across multiple GPUs. Each GPU holds a different part of the model, resolving the memory capacity issue for the largest deep learning models with billions of parameters. This notebook uses tensor parallelism techniques which allow GPUs to work simultaneously on the same layer of a model and achieve low latency inference relative to a pipeline parallel solution.

SageMaker has rolled out DeepSpeed container which now provides users with the ability to leverage the managed serving capabilities and help to provide the un-differentiated heavy lifting.

In this notebook, we deploy the open source [FLAN-T5-XXL](https://huggingface.co/google/flan-t5-xxl) model across GPUs on a ml.g5.12xlarge instance. 

The Flan-T5 are T5 models trained on the Flan collection of datasets which include: taskmaster2, djaym7/wiki_dialog, deepmind/code_contests, lambada, gsm8k, aqua_rat, esnli, quasc and qed.

[FasterTransformer](https://github.com/NVIDIA/FasterTransformer) implements a highly optimized transformer layer for both the encoder and decoder for inference. On Volta, Turing and Ampere GPUs, the computing power of Tensor Cores are used automatically when the precision of the data and weights are FP16.

FasterTransformer is built on top of CUDA, cuBLAS, cuBLASLt and C++. We provide at least one API of the following frameworks: TensorFlow, PyTorch and Triton backend. Users can integrate FasterTransformer into these frameworks directly. For supporting frameworks, we also provide example codes to demonstrate how to use, and show the performance on these frameworks


## Licence agreement
 - View license information https://huggingface.co/models?license=license:apache-2.0 before using the model.
 - This notebook is a sample notebook and not intended for production use. Please refer to the licence at https://github.com/aws/mit-0. 

 
 


In [ ]:
!pip install sagemaker boto3 huggingface_hub --upgrade #--quiet

In [3]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
from huggingface_hub import snapshot_download
from pathlib import Path
import os

In [29]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = (
    "hf-large-model-djl/code_flan_xxl_ft"  # folder within bucket where code artifact will go
)

s3_model_prefix = (
    "hf-large-model-djl/model_flan_xxl_ft"  # folder within bucket where code artifact will go
)
region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

### Define a variable to contain the s3url of the location that has the model

In [30]:
pretrained_model_location = ('s3://sagemaker-us-east-1-425576326687/raw_weights/peft/flant5-xxl/')
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

Pretrained model will be downloaded from ---- > s3://sagemaker-us-east-1-425576326687/raw_weights/peft/flant5-xxl/


## Create SageMaker compatible Model artifact,  upload Model to S3 and bring your own inference script.

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or post-processing of the model's predictions.

However, in this notebook, we demonstrate how to deploy a model with custom inference code.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - `serving.properties` and `model.py`.

The tarball is in the following format

```
code
├──── 
│   └── serving.properties
│   └── model.py
   

```

- `serving.properties` is the configuration file that can be used to configure the model server.
- `model.py` is the script handles any requests for serving.


#### Create serving.properties 

This is a configuration file to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration.

Here is a list of settings that we use in this configuration file -
- `engine`: The engine for DJL to use. In this case, it is **FasterTransformer**.
- `option.entryPoint`: The entrypoint python file or module. This should align with the engine that is being used. 
- `option.s3url`: Set this to the URI of the Amazon S3 bucket that contains the model. 

If you want to download the model from huggingface.co, you can set `option.modelid`. The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co. 
- `option.tensor_parallel_degree`: Set to the number of GPU devices over which the model needs to be partitioned. This parameter also controls the number of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine, and we are creating 8 partitions then we will have 1 worker per model to serve the requests.

For more details on the configuration options and an exhaustive list, you can refer the documentation - https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html.


In [31]:
!rm -r code_fasterformer_flant5
!mkdir -p code_fasterformer_flant5

In [32]:
%%writefile code_fasterformer_flant5/serving.properties
# engine = FasterTransformer
option.tensor_parallel_degree = 1
engine=Python
option.s3url = {{s3url}}
#option.model_id=google/flan-t5-small_LORA_SEQ_2_SEQ_LM

Writing code_fasterformer_flant5/serving.properties


In [33]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("code_fasterformer_flant5/serving.properties").open().read())
Path("code_fasterformer_flant5/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize code_fasterformer_flant5/serving.properties | cat -n

     1	# engine = FasterTransformer
     2	option.tensor_parallel_degree = 1
     3	engine=Python
     4	option.s3url = s3://sagemaker-us-east-1-425576326687/raw_weights/peft/flant5-xxl/
     5	#option.model_id=google/flan-t5-small_LORA_SEQ_2_SEQ_LM


In [34]:
#!pip install peft@https://github.com/huggingface/peft.git@v0.2.0


In [35]:
%%writefile code_fasterformer_flant5/requirements.txt
peft
#peft@git+https://github.com/huggingface/peft.git


Writing code_fasterformer_flant5/requirements.txt


In [36]:
!pygmentize code_fasterformer_flant5/requirements.txt | cat -n

     1	peft
     2	#peft@git+https://github.com/huggingface/peft.git


In [37]:
import fastertransformer as ft



In [675]:
import torch
torch.is_tensor(None)

False

#### Create a model.py

In [678]:
%%writefile code_fasterformer_flant5/model.py
from djl_python import Input, Output
from peft import PeftModel, PeftConfig
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
#import fastertransformer as ft
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    T5Tokenizer,
    T5ForConditionalGeneration,
    GenerationConfig,
    pipeline
)
import os
import logging
import math
import traceback
import deepspeed


peft_pipeline = None
model = None
tokenizer = None
ft_model = None

def load_model(properties):
    model_name = "google/flan-t5-xxl"
    tensor_parallel_degree = properties.get("tensor_parallel_degree", 1)
    pipeline_parallel_degree = 1
    model_location = properties["model_dir"]
    if "model_id" in properties:
        model_location = properties["model_id"]
    logging.info(f"Loading model in {model_location}")


    # tokenizer = T5Tokenizer.from_pretrained(model_location)
    # dtype = "fp32"
    # model = ft.init_inference(
    #     model_location, tensor_parallel_degree, pipeline_parallel_degree, dtype
    # )
    # return model, tokenizer



    # load base LLM model and tokenizer
    peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

    # Load the LoRA/PEFT model
    peft_model = PeftModel.from_pretrained(peft_model_base, f'{model_location}/', device_map="auto")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    peft_model.eval()
    peft_model.to(device)
    
    total_params = sum(p.numel() for p in peft_model.parameters())

    logging.info(f"Peft model loaded:total_params={total_params}::")

    peft_pipe = pipeline(task="summarization", model=peft_model, tokenizer=tokenizer, device=device)
    
    ft_model = None
    try:
        logging.info("Trying to create the FasterTransformer Pipeline" )
        dtype = "fp32"
        # -ft_model = ft.init_inference(model_location, tensor_parallel_degree, pipeline_parallel_degree, dtype)
        logging.info(f"Starting DeepSpeed init with TP={pipeline_parallel_degree}")
        
        # load base LLM model and tokenizer
        ds_peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", device_map="auto") # - , device_map="auto", balanced_low_0 fails division by 0
        ft_model = PeftModel.from_pretrained(ds_peft_model_base, f'{model_location}/', device_map="auto")  #- , device_map="auto", balanced_low_0 fails division by 0
        ft_model.eval()
        ft_model.to(device)
        # - peft pipe fails
        # - DEEP SPEED does not support FLAN T5
        # ft_model = deepspeed.init_inference(
        #     PeftModel.from_pretrained(ds_peft_model_base, f'{model_location}/'),  #- , device_map="auto"
        #     mp_size=tensor_parallel_degree,
        #     dtype=dtype, #model.dtype,
        #     replace_method="auto",
        #     replace_with_kernel_inject=True,
        # )
        logging.info(f"Success !! ACCELERATE init with TP={pipeline_parallel_degree}:model={type(ft_model)}::model={ft_model}::")
    except:
        err_str = traceback.format_exc()
        logging.info(f"error in Init():ACCELERATE:DeepSpeed::FasterTransformer::{err_str}")  
    
    return peft_pipe, peft_model, tokenizer, ft_model



def run_pipeline(peft_pipeline, tokenizer, data):
    try:
        logging.info(f'run_pipeline():Starting')
        input_sentences = data["inputs"]
        params = data["parameters"]
        logging.info(f"run_pipeline():PEFT:loaded:{type(input_sentences)}::input_sentences={input_sentences}::params={params}::")
        
        prompt = f'{input_sentences}'
        # - max_new_tokens` and `max_length` h
        params = {'max_length':200, 'num_beams':1}
        peft_model_text_output = peft_pipeline(prompt, **params)

        logging.info(f'run_pipeline():PEFT:Prompt:\n--------------------------\n{prompt}\n--------------------------')
        logging.info(f'run_pipeline():PEFT: model summary: {peft_model_text_output}')
        result = {"outputs": peft_model_text_output}
        return result
    
    except:
        err_str = traceback.format_exc()
        logging.info(f"error in run_pipeline()::{err_str}")  
        return {"outputs": err_str}
        
def run_model_inference(model, tokenizer, data):
    
    try:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        input_sentences = data["inputs"]
        params = data["parameters"]
        logging.info(f"run_model_inference():PEFT:loaded:{type(input_sentences)}::input_sentences={input_sentences}::params={params}::")

        prompt = f'Summarize the following conversation.\n\n{input_sentences}\n\nSummary:'
        input_ids = tokenizer(prompt, return_tensors="pt").to(device).input_ids

        model_outputs = model.generate(input_ids=input_ids) #, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)) 
        # - input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)) #gen_config)
        logging.info(f"run_model_inference():PEFT:OUTPUT:peft_model_outputs:{model_outputs}:")

        model_text_output = tokenizer.batch_decode(model_outputs, skip_special_tokens=True)
        logging.info(f"run_model_inference():PEFT:model_text_output={model_text_output}:")

        result = {"outputs": model_text_output}
        return result
    except:
        err_str = traceback.format_exc()
        logging.info(f"error in hardcode::{err_str}")  
        return {"outputs": err_str}
    
def run_ds_model(model, tokenizer, data):
    
    try:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        input_sentences = data["inputs"]
        params = data["parameters"]
        logging.info(f"run_ds_model():PEFT:loaded:{type(input_sentences)}::input_sentences={input_sentences}::params={params}::")

        #tokenizer.pad_token = tokenizer.eos_token
        input_ids = tokenizer(input_sentences, return_tensors="pt").to(device).input_ids
        
        model_outputs = None
        error_list=[]
        
        # -- self.module.generate(*inputs, **kwargs) 
        try:
            model_outputs = model.generate(input_ids)
            logging.info(f"run_ds_model():PEFT:a:{model_outputs}:")
        except:
            error_list.append(f"a:error:{traceback.format_exc()}:")
            pass

        # -- self.module.generate(*inputs, **kwargs) 
        try:
            model_outputs = model.generate(*input_ids)
            logging.info(f"run_ds_model():PEFT:ab:{model_outputs}:")
        except:
            error_list.append(f"ab:error:{traceback.format_exc()}:")
            pass
        
        # -- self.module.generate(*inputs, **kwargs) 
        try:
            model_outputs = model.generate(*input_ids, **params)
            logging.info(f"run_ds_model():PEFT:acc:{model_outputs}:")
        except:
            error_list.append(f"acc:error:{traceback.format_exc()}:")
            pass
        
        # -- self.module.generate(*inputs, **kwargs) -- this worked
        try:
            model_outputs = model.generate(inputs=input_ids, **params)
            logging.info(f"run_ds_model():PEFT:addd:{model_outputs}:")
        except:
            error_list.append(f"addd:error:{traceback.format_exc()}:")
            pass
        
        # -- self.module.generate(*inputs, **kwargs)  -- this worked 
        try:
            model_outputs = model.generate(inputs=input_ids)
            logging.info(f"run_ds_model():PEFT:aeee:{model_outputs}:")
        except:
            error_list.append(f"aeee:error:{traceback.format_exc()}:")
            pass

        try:
            model_outputs = model.generate(input_ids, params)
            logging.info(f"run_ds_model():PEFT:b:{model_outputs}:")
        except:
            error_list.append(f"b:error:{traceback.format_exc()}:")
            pass
        
        try:
            model_outputs = model.generate(input_ids, **params)
            logging.info(f"run_ds_model():PEFT:c:{model_outputs}:")
        except:
            error_list.append(f"c:error:{traceback.format_exc()}:")
            pass

        try:
            example = "My name is Philipp and I"
            input_ids = tokenizer(input_sentences, return_tensors="pt").to(device).input_ids
            model_outputs = model.generate(input_ids, do_sample=True, max_length=100)
            logging.info(f"run_ds_model():PEFT:d:{model_outputs}:")
        except:
            error_list.append(f"d:error:{traceback.format_exc()}:")
            pass
        
        try:
            model_outputs = model.generate(**input_ids, **params)
            logging.info(f"run_ds_model():PEFT:f:{model_outputs}:")
        except:
            error_list.append(f"f:error:{traceback.format_exc()}:")
            pass
        

        try:
            model_outputs = model.module.generate(input_ids, **params)
            logging.info(f"run_ds_model():PEFT:e:{model_outputs}:")
        except:
            error_list.append(f"e:error:{traceback.format_exc()}:")
            pass
        
        #logging.info(f"run_ds_model:ALL:ERRORS:{error_list}:")
        
        logging.info(f"run_ds_model():PEFT:OUTPUT:peft_model_outputs:{model_outputs}:")
        outputs="error"
        if torch.is_tensor(model_outputs):
            outputs = tokenizer.batch_decode(model_outputs, skip_special_tokens=True)

        result = {"outputs": outputs}
        return result
    
    except:
        err_str = traceback.format_exc()
        logging.info(f"error in run_ds_model::{err_str}")  
        return {"outputs": err_str}
    

def handle(inputs: Input):
    """
    inputs: Contains the configurations from serving.properties
    """
    global peft_pipeline, model, tokenizer, ft_model
    # - peft_pipe, peft_model, tokenizer
    if not peft_pipeline:
        peft_pipeline, model, tokenizer, ft_model = load_model(inputs.get_properties())

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        return None

    
    data = inputs.get_as_json()
    
    outputs = run_pipeline(peft_pipeline, tokenizer, data)
    
    run_model_inference(model, tokenizer, data) 
    
    run_ds_model(ft_model, tokenizer, data)
                      
    return Output().add_as_json(outputs)



Overwriting code_fasterformer_flant5/model.py


**Image URI for the DJL container is being used here**

In [679]:
#- 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-fastertransformer5.3.0-cu117
inference_image_uri = image_uris.retrieve(
    framework="djl-fastertransformer", region=sess.boto_session.region_name, version="0.22.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Defaulting to the only supported framework/algorithm version: 0.21.0. Ignoring framework/algorithm version: 0.22.0.


Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-fastertransformer5.3.0-cu117


In [680]:
# -- DEEPSPEED -- 

# inference_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/djl-ds:latest"
#inference_image_uri = (f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.20.0-deepspeed0.7.5-cu116")

inference_image_uri =f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117


### -- LATEST Final image

In [681]:
#inference_image_uri = image_uris.retrieve(framework="djl-deepspeed", region=sess.boto_session.region_name, version="0.21.0")
# - 763104351884.dkr.ecr.region.amazonaws.com/djl-inference:0.21.0-fastertransformer5.3.0-cu117
# - deepjavalibrary/djl-serving:0.22.1-fastertransformer
# - docker pull deepjavalibrary/djl-serving:0.21.0-pytorch-inf2

print(f"Image going to be used is ---- > {inference_image_uri}")


Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117


In [682]:
#-public.ecr.aws/j5o1q8s5/neuron-beta-sd:latest

**Create the Tarball and then upload to S3 location**

In [683]:
!rm model.tar.gz
!cd code_fasterformer_flant5 && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz code_fasterformer_flant5

code_fasterformer_flant5/
code_fasterformer_flant5/model.py
code_fasterformer_flant5/requirements.txt
code_fasterformer_flant5/serving.properties


In [684]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-425576326687/hf-large-model-djl/code_flan_xxl_ft/model.tar.gz


In [685]:
!aws s3 ls s3://sagemaker-us-east-1-425576326687/hf-large-model-djl/code_flan_xxl_ft/model.tar.gz

2023-05-01 22:53:38       2573 model.tar.gz


### To create the end point the steps are:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.g5.24xlarge 
    
    b) ContainerStartupHealthCheckTimeoutInSeconds is 2400 to ensure health check starts after the model is ready    
3. Create the end point using the endpoint config created    
    

#### Create the Model
Use the image URI for the DJL container and the s3 location to which the tarball was uploaded.

The container downloads the model into the `/tmp` space on the instance because SageMaker maps the `/tmp` to the Amazon Elastic Block Store (Amazon EBS) volume that is mounted when we specify the endpoint creation parameter VolumeSizeInGB. It leverages `s5cmd`(https://github.com/peak/s5cmd) which offers a very fast download speed and hence extremely useful when downloading large models.

For instances like p4dn, which come pre-built with the volume instance, we can continue to leverage the `/tmp` on the container. The size of this mount is large enough to hold the model.


In [686]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"flan-xxl-fastertransformer")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri, "ModelDataUrl": s3_code_artifact},
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

flan-xxl-fastertransformer-2023-05-01-22-53-41-287
Created Model: arn:aws:sagemaker:us-east-1:425576326687:model/flan-xxl-fastertransformer-2023-05-01-22-53-41-287


In [687]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.12xlarge",
            "InitialInstanceCount": 1,
            "ContainerStartupHealthCheckTimeoutInSeconds": 180, #600,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:425576326687:endpoint-config/flan-xxl-fastertransformer-2023-05-01-22-53-41-287-config',
 'ResponseMetadata': {'RequestId': 'bb1d49bb-1ef7-4a5a-84d8-44f221f4bc4d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'bb1d49bb-1ef7-4a5a-84d8-44f221f4bc4d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '138',
   'date': 'Mon, 01 May 2023 22:53:42 GMT'},
  'RetryAttempts': 0}}

In [688]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-1:425576326687:endpoint/flan-xxl-fastertransformer-2023-05-01-22-53-41-287-endpoint


### This step can take ~ 10 min or longer so please be patient

In [689]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:425576326687:endpoint/flan-xxl-fastertransformer-2023-05-01-22-53-41-287-endpoint
Status: InService


#### While you wait for the endpoint to be created, you can read more about:
- [Deep Learning containers for large model inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-dlc.html)
- [FasterTransformer](https://github.com/NVIDIA/FasterTransformer)

#### Leverage the Boto3 to invoke the endpoint. 

This is a generative model so we pass in a Text as a prompt and the Model will complete the sentence and return the results.

You can pass a batch of prompts as input to the model. This is done by setting `inputs` to the list of prompts. The model then returns a result for each prompt. The text generation can be configured using appropriate parameters. These `parameters` need to be passed to the endpoint as a dictionary of `kwargs`.

The below code sample illustrates the invocation of the endpoint using prompts and also sets some parameters.

Here's a list of default arguments that's used by the model for inference. You can pass specific values based on the use case - 
```
default_args = dict(
            inputs_embeds=None,
            beam_width=1,
            max_seq_len=200,
            top_k=1,
            top_p=0.0,
            beam_search_diversity_rate=0.0,
            temperature=1.0,
            len_penalty=0.0,
            repetition_penalty=1.0,
            presence_penalty=None,
            min_length=0,
            random_seed=0,
            is_return_output_log_probs=False,
            is_return_cum_log_probs=False,
            is_return_cross_attentions=False,
            bad_words_list=None,
            stop_words_list=None
        )
```


In [690]:
%%time

input_sentences=["Amazon.com is the best ",]
prompt = f'Summarize the following conversation.\n\n{input_sentences}\n\nSummary:'

smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            #"inputs": ["Amazon.com is the best ", "Large model inference is"],
            "inputs": prompt, # "Large model inference is"],
            "parameters": {'max_new_tokens':200, 'num_beams':1},
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

CPU times: user 10.4 ms, sys: 0 ns, total: 10.4 ms
Wall time: 2.06 s


'{\n  "outputs":[\n    {\n      "summary_text":"You can find the best deals on Amazon.com. You can also use the Amazon.co.uk website to find the cheapest deals."\n    }\n  ]\n}'

### Examples of prompt engineering for 'Zero Shot' NLP tasks
Here are some examples of how you can use the deployed model for zero-shot NLP tasks.

#### Common Sense reasoning

In [35]:
%%time
# -- Greedy generation
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": [
                "The world cup has kicked off in Los Angeles, United States.\n\nBased on the paragraph above can we conclude that \”The world cup takes place in United States.\”?\n\n[\”yes\”, \”no\”]"
            ],
            "parameters": {"temperature": 0.7},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

CPU times: user 3.2 ms, sys: 309 µs, total: 3.51 ms
Wall time: 1.47 s


'{\n  "outputs":[\n    {\n      "summary_text":"yes, no, \\"no\\", \\"no\\"., and \\"n\\".\\"."\n    }\n  ]\n}'

#### Text / Sentiment classification

In [ ]:
%%time
# -- Greedy generation
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": [
                "Review:\nThis moive is so great and once again dazzles and delights us\nIs this movie review sentence negative or positive?\nOPTIONS:\n-positive \n-negative"
            ],
            "parameters": {},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

#### Translation

In [1]:
%%time
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": ["My name is Arthur\n\nTranslate to German"],
            "parameters": {"temperature": 0.7},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

NameError: name 'smr_client' is not defined

#### Article generation

In [3]:
%%time
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": [
                "Title: \”University has new facility coming up“\\nGiven the above title of an imaginary article, imagine the article.\n"
            ],
            "parameters": {"max_seq_len": 200, "temperature": 0.7},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

NameError: name 'smr_client' is not defined

#### Abstractive Question Answering

In [ ]:
context = """
Customer: Hi there, I'm having a problem with my iPhone.
Agent: Hi! I'm sorry to hear that. What's happening?
Customer: The phone is not charging properly, and the battery seems to be draining very quickly. I've tried different charging cables and power adapters, but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to Settings, then Battery, and see if there are any apps that are using up a lot of battery life?
Customer: Yes, there are some apps that are using up a lot of battery.
Agent: Okay, try force quitting those apps by swiping up from the bottom of the screen and then swiping up on the app to close it.
Customer: I did that, but the issue is still there.
Agent: Alright, let's try resetting your iPhone's settings to their default values. This won't delete any of your data. Go to Settings, then General, then Reset, and then choose Reset All Settings.
Customer: Okay, I did that. What's next?
Agent: Now, let's try restarting your iPhone. Press and hold the power button until you see the "slide to power off" option. Slide to power off, wait a few seconds, and then turn your iPhone back on.
Customer: Alright, I restarted it, but it's still not charging properly.
Agent: I see. It looks like we need to run a diagnostic test on your iPhone. Please visit the nearest Apple Store or authorized service provider to get your iPhone checked out.
Customer: Do I need to make an appointment?
Agent: Yes, it's always best to make an appointment beforehand so you don't have to wait in line. You can make an appointment online or by calling the Apple Store or authorized service provider.
Customer: Okay, will I have to pay for the repairs?
Agent: That depends on whether your iPhone is covered under warranty or not. If it is, you won't have to pay anything. However, if it's not covered under warranty, you will have to pay for the repairs.
Customer: How long will it take to get my iPhone back?
Agent: It depends on the severity of the issue, but it usually takes 1-2 business days.
Customer: Can I track the repair status online?
Agent: Yes, you can track the repair status online or by calling the Apple Store or authorized service provider.
Customer: Alright, thanks for your help.
Agent: No problem, happy to help. Is there anything else I can assist you with?
Customer: No, that's all for now.
Agent: Alright, have a great day and good luck with your iPhone!
"""

#### Q1

In [ ]:
query = (
    "What troubleshooting steps were suggested to the customer to fix their iPhone charging issue?"
)

In [ ]:
%%time
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": [f"{context}\n{query}"],
            "parameters": {"max_seq_len": 256, "temperature": 0.4, "top_k": 0, "top_p": 0.7},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

#### Q2

In [ ]:
query = "Was resetting the iPhone to its default settings able to solve the charging issue and battery drain problem?"

In [ ]:
%%time
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": [f"{context}\n{query}"],
            "parameters": {"max_seq_len": 256, "temperature": 0.4, "top_k": 0, "top_p": 0.7},
            "padding": True,
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

## Clean Up

In [676]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '4424c914-430d-4a53-a25b-17cf3d99e9c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4424c914-430d-4a53-a25b-17cf3d99e9c1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 01 May 2023 22:53:18 GMT'},
  'RetryAttempts': 0}}

In [677]:
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': 'f5e0157f-e067-4569-8178-53d044059f06',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f5e0157f-e067-4569-8178-53d044059f06',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 01 May 2023 22:53:19 GMT'},
  'RetryAttempts': 0}}